In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import clear_output 

objc[6286]: Class CaptureDelegate is implemented in both /Users/vaisakh/miniconda3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1080c8860) and /Users/vaisakh/miniconda3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x156fb25a0). One of the two will be used. Which one is undefined.
objc[6286]: Class CVWindow is implemented in both /Users/vaisakh/miniconda3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x103024a68) and /Users/vaisakh/miniconda3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x156fb25f0). One of the two will be used. Which one is undefined.
objc[6286]: Class CVView is implemented in both /Users/vaisakh/miniconda3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x103024a90) and /Users/vaisakh/miniconda3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x156fb2618). One of the two will be used. Which one is undefined.
objc[6286]: Class CVSlider is implemented in both /Users/vaisa

In [2]:
model = load_model("mp_last_large_map_final")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [3]:
def preprocess(landmark):
    processed_landmarks = []
    relative_landmarks= []
    for i in landmark[0].landmark:
        processed_landmarks.append([i.x, i.y])

    for i in processed_landmarks:
        relative_landmarks.append([
            i[0]-processed_landmarks[0][0],
            i[1]-processed_landmarks[0][1],
        ])
    
    flatten = []
    for i in relative_landmarks:
        flatten.append(i[0])
        flatten.append(i[1])
    
    maxVal = max(list(map(abs, flatten)))
    def norm(n): return n/maxVal
    normalised = list(map(norm, flatten))

    return normalised

In [4]:
encoding_maps = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
       'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 
       'U', 'V', 'W', 'X', 'Y', 'Z']

In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = cv2.VideoCapture(1)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
    # while True:
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                        )

            res = preprocess(results.multi_hand_landmarks)
            prob = model.predict([res])
            clear_output()
            # print("prob: ", prob)
            pred = np.argmax(prob)
            maxi = prob[0][pred]
            tresh = .90
            sign = encoding_maps[pred]
            print("res:", sign, "max - prob:", maxi)
            if maxi > tresh:
                image = cv2.putText(image, sign, org=(50, 50), 
                                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                                    fontScale=1, color=(255,255,255), 
                                    thickness=2)

        cv2.imshow('Hand Tracking', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

prob:  [[4.22902360e-07 2.71369035e-22 1.02099499e-15 1.05216879e-36
  1.43851951e-18 1.33305378e-12 1.37856425e-14 4.32008686e-29
  2.41157090e-06 1.92227958e-08 4.10506412e-18 0.00000000e+00
  5.70370140e-10 3.06359965e-10 1.04813775e-08 3.77274684e-23
  1.27894873e-32 6.59774234e-26 8.88966625e-08 1.12610437e-07
  5.36655443e-06 8.65611707e-24 2.53625745e-38 1.61195110e-25
  6.98658145e-12 9.99991536e-01 3.20750574e-27]]
res: Y max - prob: 0.99999154


-1